# Python Intro - Final Exercise


In [140]:
import pyodbc
import pandas as pd
import numpy as np

In [141]:
### In windows, Using a ODBC DNS (predefined connection name)
conn =  pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-VA3UHPO\SQLEXPRESS;"
                      "Database=COlleage;"
                      "Trusted_Connection=yes;")



In [217]:
### Get the students table
students = pd.read_sql("SELECT * FROM dbo.students;",conn)
students.head()
courses = pd.read_sql("SELECT * FROM dbo.courses;",conn)
classrooms = pd.read_sql("SELECT * FROM dbo.classrooms;",conn)
departments = pd.read_sql("SELECT * FROM dbo.departments;",conn)
teachers = pd.read_sql("SELECT * FROM dbo.teachers;",conn)

print(teachers)

    TeacherId  FirstName      LastName Gender
0           1     Kaylea      Monzingo      F
1           2     Amanda       Dworkin      F
2           3     Jordyn         Baron      F
3           4  Katherine        Dibari      F
4           5    Heather       Metcalf      F
5           6    Madison         Batan      F
6           7    Cortnee  Kinderknecht      F
7           8   Caroline       Knuesel      F
8           9   Katerina         Young      F
9          10  Mackenzie         Olsen      F
10         11    Kaitlin        Delude      F
11         12    Mikayla       Russell      F
12         13  Alexander         Watts      M
13         14    Jeffrey        Batzel      M
14         15      Conor    Mckittrick      M
15         16     Robert      Aivazian      M
16         17     Tanner    Fitzsimons      M


# Questions
 
 
## Q1. Count the number of students on each department

In [283]:
## Get the students by Department
df1 = pd.merge(classrooms,courses,on='CourseId')
df1 = pd.merge(df1,departments,on='DepartmentId')

gb_df1=df1.groupby('DepartmentName').agg({'StudentId':'nunique'})

gb_df1

,StudentId
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [293]:
# Get the English courses per student 

df2 = pd.merge(classrooms,courses,on='CourseId')
df2 = df2[df2.DepartmentId == 1]

gb_df2 = df2.groupby('CourseId').agg({'StudentId':'count'})
print(gb_df2)
tot=len(df2.drop_duplicates('StudentId'))
print("-------------------")
print("total           ",tot)


          StudentId
CourseId           
1                32
2                24
3                38
-------------------
total            84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [315]:
df3= pd.merge(classrooms,courses,on='CourseId')
df3= df3[df3.DepartmentId == 2]

gb_df3 = df3.groupby('CourseName').agg({'StudentId' : 'count'})
gb_df3['clasroom_size']='Small classrooms'
gb_df3.loc[(gb_df3['StudentId']>=22),'clasroom_size']='Big_Clasrooms'
gb_df4=gb_df3.groupby('clasroom_size').agg('count')
gb_df4


,StudentId
clasroom_size,
Big_Clasrooms,10
Small classrooms,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [146]:
df4=students.groupby('Gender').agg({'StudentId':'count'})
df4

,StudentId
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [211]:
# Merge df:
df = pd.merge(classrooms,courses,on='CourseId', how='left')
df = pd.merge(df,departments,on='DepartmentId', how='left')
df = pd.merge(df,students,on='StudentId', how='left')
print(df.shape[0])

courses_count_df = df.groupby('CourseName')['StudentId'].count().reset_index()
courses_count_df = courses_count_df.rename(columns={'StudentId': 'course_count'})

course_gb = df.groupby(by=['CourseName','Gender'])['StudentId'].count().reset_index()
course_gb = pd.merge(course_gb,courses_count_df,on='CourseName', how='left')
course_gb['gender_prec'] = course_gb.apply(lambda x: (x['StudentId']/x['course_count']),axis=1)
over_gender_prec = course_gb[course_gb['gender_prec']>0.7].reset_index(drop=True)
over_gender_prec.head(3)

courses = list(over_gender_prec['CourseName'])
print("courses the percentage of male/female students is over 70%:")
print(courses)

754
courses the percentage of male/female students is over 70%:
['Genetics', 'Sculpture', 'Tenis']


## Q6. For each department, how many students passed with a grades over 80?

In [213]:

df = df.dropna()

df = df.sort_values(by='degree', ascending=False).reset_index(drop=True)
df = df.drop_duplicates(subset=['CourseId','StudentId'], keep='first')

df['above_80'] = df['degree'].apply(lambda x: 1 if x>80 else 0)

gb_df = df.groupby(by='DepartmentName').agg({'StudentId':'nunique','above_80':'sum'}).reset_index()

gb_df['prec(%)'] = gb_df.apply(lambda x: round((x['above_80']/x['StudentId'])*100,1),axis=1)
gb_df



,DepartmentName,StudentId,above_80,prec(%)
0,Arts,137,68,49.6
1,English,84,43,51.2
2,Science,200,169,84.5
3,Sport,81,41,50.6


## Q7. For each department, how many students passed with a grades under 60?

In [215]:
df2['student60']=0
df2.loc[(df2['degree']<60),'student60']=1
gb_df2=df2.groupby(['DepartmentName']).agg({'StudentId':'nunique','student60':'sum'})
gb_df2['prec(%)'] = gb_df2.apply(lambda x: round((x['student60']/x['StudentId'])*100,1),axis=1)
gb_df2

,StudentId,student60,prec(%)
DepartmentName,,,
Arts,137,3,2.2
English,84,2,2.4
Science,200,7,3.5
Sport,81,1,1.2


## Q8. Rate the teachers by their average student's grades (in descending order).

In [229]:
df8 = pd.merge(classrooms,courses,on='CourseId')
df8 = pd.merge(df8,teachers,on='TeacherId')

gd_df8 = df8.groupby(['FirstName','LastName']).agg({"degree":"mean"})
gb_df8 = gd_df8.sort_values(['degree'] ,ascending = False)
gb_df8

,,degree
FirstName,LastName,
Jordyn,Baron,80.729494
Heather,Metcalf,80.440242
Kaylea,Monzingo,79.365537
Conor,Mckittrick,78.699492
Tanner,Fitzsimons,78.443395
Mackenzie,Olsen,78.383348
Madison,Batan,77.743575
Katerina,Young,77.406852
Mikayla,Russell,77.134323


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [249]:
courses.rename(columns={'DepartmentID' : 'DepartmentId'}, inplace=True)
df9 = pd.merge(departments,courses,on='DepartmentId')
df9 = pd.merge(df9,teachers,on='TeacherId')
df9 = pd.merge(df9,classrooms,on='CourseId')
df9
gb_df9 = df9.groupby(['CourseId','CourseName','DepartmentName','FirstName','LastName']).agg({'studentId':'nunique'})
#gb_df9

SpecificationError: nested renamer is not supported

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [280]:
df10 = pd.merge(students,classrooms,on='StudentId', how='left')
df10 = pd.merge(df10,courses,on='CourseId', how='left')
df10.loc[(df10['DepartmentId']==1),'English_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==2),'Science_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==3),'Arts_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==4),'Sports_degree']=df10['degree']

gb_df10 = df10.groupby(['StudentId','FirstName','LastName']).agg(
{'CourseId' : 'nunique', 'degree' : 'mean' ,'English_degree' : 'mean',
'Science_degree' : 'mean','Arts_degree' : 'mean', 'Sports_degree' : 'mean' })
gb_df10

,,,CourseId,degree,English_degree,Science_degree,Arts_degree,Sports_degree
StudentId,FirstName,LastName,,,,,,
1,Alexandria,Heller,4,81.337515,NaN,79.557508,86.677535,NaN
2,Cassie,Liska,5,79.137364,NaN,78.867115,79.542738,NaN
3,Hayley,Frank,3,78.793700,83.972245,76.204427,NaN,NaN
4,Maxwell,Kotch,3,71.481003,70.929801,74.157066,69.356144,NaN
5,Dylan,Boehm,3,76.718650,NaN,100.000000,64.494845,65.661106
...,...,...,...,...,...,...,...,...
276,Dylan,Bannister,3,76.784735,NaN,76.784735,NaN,NaN
277,Bryan,Medina,6,80.482335,NaN,81.298837,78.849330,NaN
278,Sydney,Blas,3,75.359189,NaN,72.578017,NaN,76.749775
